In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [ ]:
import addict
import numpy as np
import matplotlib.pyplot as plt

import celeri

# Read data and pre-process

In [ ]:
COMMAND_FILE_NAME = "../data/config/japan_config.json"

model = celeri.build_model(COMMAND_FILE_NAME)
operators = celeri.build_operators(model, tde=True, eigen=True)

# Solve bounded KL+QP problem

In [ ]:
# Get QP bounds as inequality constraints
qp_inequality_constraints_matrix, qp_inequality_constraints_data_vector = (
    celeri.get_qp_all_inequality_operator_and_data_vector(
        model, operators, operators.index
    )
)

# QP solve
opts = {"show_progress": True}
solution_qp = celeri.lsqlin_qp(
    operators.full_dense_operator * np.sqrt(operators.weighting_vector[:, None]),
    operators.data_vector * np.sqrt(operators.weighting_vector),
    0,
    qp_inequality_constraints_matrix,  # Inequality matrix
    qp_inequality_constraints_data_vector,  # Inequality data vector
    None,
    None,
    None,
    None,
    None,
    opts,
)

estimation_qp = celeri.solve.Estimation(
    data_vector=operators.data_vector,
    weighting_vector=operators.weighting_vector,
    operator=operators.full_dense_operator,
    state_vector=np.array(solution_qp["x"]).ravel(),
    model=model,
    operators=operators,
    state_covariance_matrix=None,
)
celeri.write_output(
    model.config, estimation_qp, model.station, model.segment, model.block, model.meshes
)

# Plot KL+QP estimation

In [ ]:
celeri.plot_estimation_summary(model, estimation_qp)